In [36]:
import random

import cassiopeia as cass
from cassiopeia import Summoner

cass.set_riot_api_key("RGAPI-7fb0310d-4d1c-458d-923d-50334d25dca1")  # This overrides the value set in your configuration/settings.

def print_summoner(name: str, region: str):
    summoner = Summoner(name=name, region=region)
    print("Name:", summoner.name)
    for i in range(2, 6): 
        print("Match History:", summoner.match_history[i].id)
        print("Match Time:", summoner.match_history[i].creation)
    print("ID:", summoner.id)
    print("Account ID:", summoner.account_id)
    print("Level:", summoner.level)
    print("Revision date:", summoner.revision_date)
    print("Profile icon ID:", summoner.profile_icon.id)
    print("Profile icon name:", summoner.profile_icon.name)
    print("Profile icon URL:", summoner.profile_icon.url)
    print("Profile icon image:", summoner.profile_icon.image)

print_summoner("Djruhnke", "NA")
    
# if __name__ == "__main__":
#     print_summoner("Kalturi", "NA")

# cass.set_riot_api_key("YOUR_KEY")  # This overrides the value set in your configuration/settings.

# djruhnke = cass.Summoner(name="Djruhnke", region="NA")
# good_with = kalturi.champion_masteries.filter(lambda cm: cm.level >= 6)
# print([cm.champion.name for cm in good_with])

# At the time of writing this, this prints:
# ["Vel'Koz", 'Blitzcrank', 'Braum', 'Lulu', 'Sejuani']

Name: Djruhnke
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/O6bdU9QzjLRf2ryY3eEo1sn3iAVmZOfiHo9QfF0IXwktCCdreml2b8ZoBvkcYFKCWY1rSvuP9MZmIw/ids?beginIndex=0&endIndex=100
Match History: NA1_4345049839
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/O6bdU9QzjLRf2ryY3eEo1sn3iAVmZOfiHo9QfF0IXwktCCdreml2b8ZoBvkcYFKCWY1rSvuP9MZmIw/ids?beginIndex=0&endIndex=100
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4345049839
Match Time: 2022-06-17T02:26:48+00:00
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/O6bdU9QzjLRf2ryY3eEo1sn3iAVmZOfiHo9QfF0IXwktCCdreml2b8ZoBvkcYFKCWY1rSvuP9MZmIw/ids?beginIndex=0&endIndex=100
Match History: 4343006055
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/O6bdU9QzjLRf2ryY3eEo1sn3iAVmZOfiHo9QfF0IXwktCCdreml2b8ZoBvkcYFKCWY1rSvuP9MZmIw/ids?beginIndex=0&endIndex=100
Match Time: 2022-06-15T04:42:52+00:00
Making call: https://

In [33]:
import random
from sortedcontainers import SortedList
import arrow

from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Queue, Patch

cass.set_riot_api_key("RGAPI-7fb0310d-4d1c-458d-923d-50334d25dca1")

def filter_match_history(summoner, patch):
    end_time = patch.end
    if end_time is None:
        end_time = arrow.now()
    match_history = MatchHistory(
        puuid=summoner.puuid,
        queue=Queue.aram,
        begin_time=patch.start,
        end_time=end_time,
        continent="AMERICAS"
    )
    return match_history


def collect_matches():
    initial_summoner_name = "Djruhnke"
    region = "NA"

    summoner = Summoner(name=initial_summoner_name, region=region)
    patch = Patch.from_str("12.10", region=region)

    unpulled_summoner_ids = SortedList([summoner.id])
    pulled_summoner_ids = SortedList()

    unpulled_match_ids = SortedList()
    pulled_match_ids = SortedList()

    while unpulled_summoner_ids:
        # Get a random summoner from our list of unpulled summoners and pull their match history
        new_summoner_id = random.choice(unpulled_summoner_ids)
        new_summoner = Summoner(id=new_summoner_id, region=region)
        matches = filter_match_history(new_summoner, patch)
        unpulled_match_ids.update([match.id for match in matches])
        unpulled_summoner_ids.remove(new_summoner_id)
        pulled_summoner_ids.add(new_summoner_id)

        while unpulled_match_ids:
            # Get a random match from our list of matches
            new_match_id = random.choice(unpulled_match_ids)
            new_match = Match(id=new_match_id, region=region)
            for participant in new_match.participants:
                if (
                    participant.summoner.id not in pulled_summoner_ids
                    and participant.summoner.id not in unpulled_summoner_ids
                ):
                    unpulled_summoner_ids.add(participant.summoner.id)
            # The above lines will trigger the match to load its data by iterating over all the participants.
            # If you have a database in your datapipeline, the match will automatically be stored in it.
            unpulled_match_ids.remove(new_match_id)
            pulled_match_ids.add(new_match_id)


if __name__ == "__main__":
    collect_matches()

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/O6bdU9QzjLRf2ryY3eEo1sn3iAVmZOfiHo9QfF0IXwktCCdreml2b8ZoBvkcYFKCWY1rSvuP9MZmIw/ids?beginIndex=0&endIndex=100&queue=450
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4341889093
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4339288685
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4326293419
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4336471842
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4335599751
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4335080989
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4325861903
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4339346970
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4343006055
Making call: https://americas.api.riotgames.com

TypeError: '<' not supported between instances of 'int' and 'str'

In [28]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter
from ratelimit import limits, sleep_and_retry
import arrow

cass.set_riot_api_key("RGAPI-7fb0310d-4d1c-458d-923d-50334d25dca1")

@sleep_and_retry
@limits(calls=20, period=60)
def print_newest_match(name: str, region: str):

    # Notice how this function never makes a call to the summoner endpoint because we provide all the needed data!

    summoner = Summoner(name=name, region=region)
    
    print(summoner.__dict__)

    # A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.

#     match_history = cass.get_match_history(
#         continent=summoner.region.continent,
#         puuid=summoner.puuid,
#         queue=Queue.normal_draft_fives,
#         begin_index=20,
#         end_index=120,
#     )
    
    match_history = cass.get_match_history(
        continent = summoner.region.continent, 
        region = summoner.region, 
        platform = summoner.region.platform, 
        puuid = summoner.puuid, 
        begin_index = 0, 
        end_index = 100,
    )
    
    for match in match_history:
        print("Test:", match.id)
    
    match_history = cass.get_match_history(
        continent = summoner.region.continent, 
        region = summoner.region, 
        platform = summoner.region.platform, 
        puuid = summoner.puuid, 
        begin_index = 20, 
        end_index = 100,
    )
    
    for match in match_history:
        print("Test:", match.creation)
    # match_history = summoner.match_history

    # Load the entire match history by iterating over all its elements so that we know how long it is.
    # Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
    # we need to know what version of the static data the champion should have. To avoid pulling many different
    # static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
    # ID from the match data (which it provides directly).
    champion_id_to_name_mapping = {
        champion.id: champion.name for champion in cass.get_champions(region=region)
    }
    print("Test:", len(match_history))
    print("Test:", match_history.begin_time)
    print("Test:", match_history.end_time)
    played_champions = Counter()
    for match in match_history:
        champion_id = match.participants[summoner].champion.id
        champion_name = champion_id_to_name_mapping[champion_id]
        played_champions[champion_name] += 1
        print("Test:", match.creation)
    print("Length of match history:", len(match_history))

    # Print the aggregated champion results
    print(f"Top 10 champions {summoner.name} played:")
    for champion_name, count in played_champions.most_common(10):
        print(champion_name, count)
    print()

    match = match_history[0]
    print("Match ID:", match.id)

    p = match.participants[summoner]
    print(
        "\nSince the match was created from a matchref, we only know one participant:"
    )
    # print(p.summoner.name, 'playing', p.champion.name)
    print(
        p.summoner.region,
        p.summoner.account_id,
        p.summoner.name,
        p.summoner.id,
        p.champion.id,
    )

    print("\nNow pull the full match data by iterating over all the participants:")
    for p in match.participants:
        # print(p.summoner.name, 'playing', p.champion.name)
        print(
            p.summoner.region,
            p.summoner.account_id,
            p.summoner.name,
            p.summoner.id,
            p.champion.id,
            p.stats.win,
            p.runes.keystone.name,
        )

    print(
        "\nIterate over all the participants again and note that the data is not repulled:"
    )
    for p in match.participants:
        # print(p.summoner.name, 'playing', p.champion.name)
        print(
            p.summoner.region,
            p.summoner.account_id,
            p.summoner.name,
            p.summoner.id,
            p.champion.id,
            p.stats.win,
            p.runes.keystone.name,
        )

    print("\nBlue team won?", match.blue_team.win)
    print("Red team won?", match.red_team.win)
    print("Participants on blue team:")
    for p in match.blue_team.participants:
        print(p.summoner.name, p.champion.name)

    # Print keystone and the stat runes for each player/champion
    for p in match.participants:
        print(p.champion.name, p.runes.keystone.name, *[r.name for r in p.stat_runes])


if __name__ == "__main__":
    print_newest_match(name="Djruhnke", region="NA")

{'_data': {<class 'cassiopeia.core.summoner.SummonerData'>: <cassiopeia.core.summoner.SummonerData object at 0x00000184B011AA30>}, '_lazy__region': <Region.north_america: 'NA'>, '_lazy__platform': <Platform.north_america: 'NA1'>}
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Djruhnke
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/O6bdU9QzjLRf2ryY3eEo1sn3iAVmZOfiHo9QfF0IXwktCCdreml2b8ZoBvkcYFKCWY1rSvuP9MZmIw/ids?beginIndex=0&endIndex=100
Test: NA1_4345133554
Test: NA1_4345067613
Test: NA1_4345049839
Test: NA1_4343006055
Test: NA1_4342938042
Test: NA1_4342920773
Test: NA1_4342883162
Test: NA1_4341906697
Test: NA1_4341889093
Test: NA1_4341854153
Test: NA1_4339346970
Test: NA1_4339333622
Test: NA1_4339288685
Test: NA1_4339322722
Test: NA1_4336744119
Test: NA1_4336684641
Test: NA1_4336607135
Test: NA1_4336548989
Test: NA1_4336528942
Test: NA1_4336471842
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid

In [ ]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter
from ratelimit import limits, sleep_and_retry
import arrow

summoner = Summoner(name="Djruhnke", region="NA")

def get_full_match_history(summoner):
    total_matches = 0
    fetched_records = 1
    full_history = None

    while fetched_records > 0:
        history = cass.get_match_history(
                continent = summoner.region.continent, 
                region = summoner.region, 
                platform = summoner.region.platform, 
                puuid = summoner.puuid, 
                begin_index = total_matches, 
                end_index = total_matches+100)

        if full_history is None:
            full_history = history
        else:
            full_history.extend(history)
            
        fetched_records = len(history)
        total_matches = total_matches + fetched_records

    return full_history

full_match_history = get_full_match_history(summoner)
print(f'Finished pulling {summoner.name}, they had {len(full_match_history)} games on this account')

In [5]:
import requests

# Create a python dictionary object.
data = {
    "X-Riot-Token": "RGAPI-7fb0310d-4d1c-458d-923d-50334d25dca1"
}
# Use request module to get request url and pass above dictionary object as query string.
response = requests.get("https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/O6bdU9QzjLRf2ryY3eEo1sn3iAVmZOfiHo9QfF0IXwktCCdreml2b8ZoBvkcYFKCWY1rSvuP9MZmIw/ids?start=20&count=20", headers=data)

In [32]:
test = response.content.decode('UTF-8').replace('"',"").strip("[]").split(",")

test

['NA1_4336212159',
 'NA1_4336117265',
 'NA1_4335623531',
 'NA1_4335599751',
 'NA1_4335513090',
 'NA1_4335471648',
 'NA1_4335080989',
 'NA1_4334995718',
 'NA1_4333783533',
 'NA1_4333618055',
 'NA1_4332558824',
 'NA1_4332596692',
 'NA1_4332494895',
 'NA1_4331560887',
 'NA1_4330515347',
 'NA1_4330471463',
 'NA1_4330266665',
 'NA1_4329558175',
 'NA1_4329533150',
 'NA1_4329498086']

In [35]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter
from ratelimit import limits, sleep_and_retry
import arrow

cass.set_riot_api_key("RGAPI-7fb0310d-4d1c-458d-923d-50334d25dca1")

new_match = Match(id="NA1_4336212159", region="NA")
print("Test:", new_match.creation)

test[0]

for match in test:
        new_match = Match(id=match, region="NA")
        print("Test:", new_match.creation)

Test: 2022-06-09T20:40:26+00:00
Test: 2022-06-09T20:40:26+00:00
Test: 2022-06-09T20:03:33+00:00
Test: 2022-06-09T04:43:22+00:00
Test: 2022-06-09T04:23:32+00:00
Test: 2022-06-09T03:54:07+00:00
Test: 2022-06-09T03:05:18+00:00
Test: 2022-06-08T20:50:42+00:00
Test: 2022-06-08T20:13:32+00:00
Test: 2022-06-07T21:24:24+00:00
Test: 2022-06-07T20:30:24+00:00
Test: 2022-06-06T21:14:42+00:00
Test: 2022-06-06T21:02:20+00:00
Test: 2022-06-06T19:07:50+00:00
Test: 2022-06-05T18:11:27+00:00
Test: 2022-06-04T17:50:34+00:00
Test: 2022-06-04T17:09:09+00:00
Test: 2022-06-04T16:18:12+00:00
Test: 2022-06-03T23:52:14+00:00
Test: 2022-06-03T23:11:20+00:00
Test: 2022-06-03T22:31:36+00:00
